In [6]:
import numpy as np
import pandas as pd
import scipy.special

import bokeh.io
import bokeh.plotting

bokeh.io.output_notebook()

Loading BokehJS ...

In [7]:
# make a range of polymerase copy number values to plot
P = np.floor(np.linspace(1,10000,200))

# parameters needed for p_bound
delta_e_P1 = -2.9 #kBT
delta_e_T7 = -8.1 #kBT
N_NS = 4.6E6 #bp

# compute p_bound from range of P values and parameters
p_bound_P1 = 1 / (1 + N_NS/P * np.exp(delta_e_P1))
p_bound_T7 = 1 / (1 + N_NS/P * np.exp(delta_e_T7))

In [8]:
p = bokeh.plotting.figure(
    frame_height=250,
    frame_width=350,
    x_axis_label='number of RNA polymerase molecules',
    y_axis_label='probability RNAP bound',
    y_axis_type="log",
)

p.line(
    x=P,
    y=p_bound_P1,
    line_join='bevel',
    line_width=2,
    line_color='red',
    legend_label="lac P1",
)

p.line(
    x=P,
    y=p_bound_T7,
    line_join='bevel',
    line_width=2,
    line_color='blue',
    legend_label="T7 A1"
)

p.legend.location = "bottom_right"

bokeh.io.show(p)